In [1]:
import numpy as np
import scipy.stats as stats

In [22]:
# compute probability of green zone
# prob ( -k sig < x < k sig )

TB = "\t"

# print header row
print("k" , TB , "Cheb>" , TB , "norm" + TB + "unif" + TB + "pois" + TB + "expn")

# define distributions
nn = stats.norm(loc=0,scale=1)
uu = stats.uniform(loc=0,scale=1)
pp = stats.poisson(mu=1)
ee = stats.expon(scale=1)

def compute_intg(dd,k):
    m = dd.mean()
    s = dd.std()
    return dd.cdf(m+k*s) - dd.cdf(m-k*s)

# compute probabilities
for k in [1,1.5,2,2.5,3]:

    cheby = 1 - 1/(k*k)
    intg_nn = compute_intg(nn,k)
    intg_uu = compute_intg(uu,k)
    intg_pp = compute_intg(pp,k)
    intg_ee = compute_intg(ee,k)

    print(f"{k:0.1f}",
          TB + f"{cheby:0.2f}"   + 
          TB + f"{intg_nn:0.2f}" + 
          TB + f"{intg_uu:0.2f}" + 
          TB + f"{intg_pp:0.2f}" +
          TB + f"{intg_ee:0.2f}" )
    
## In each case, Cheby gives a reliable lower bound for the probability of being within k standard deviations of the
## mean.

k 	 Cheb> 	 norm	unif	pois	expn
1.0 	0.00	0.68	0.58	0.55	0.86
1.5 	0.56	0.87	0.87	0.92	0.92
2.0 	0.75	0.95	1.00	0.98	0.95
2.5 	0.84	0.99	1.00	0.98	0.97
3.0 	0.89	1.00	1.00	1.00	0.98


In [56]:
# manually simulate a hidden markov model
# state 0 is "hot day"
# state 1 is "cold day"

# transition matrix
A = np.array([[0.6,0.5],[0.4,0.5]])

# emission matrix (probability of observations for each state)
# note that 1 ice cream is "observation #0", 2 ice creams is "observation #1" and 3 ice creams is "observation #2"
b = np.array([[0.2,0.4,0.4],[0.5,0.4,0.1]])

# assume we observed 3, then 1, then 3

# probability of observations on each day
pr_obs_day_1 = b[:,2]   # day 1 was a 3-ice cream day (observation #2)
pr_obs_day_2 = b[:,0]   # day 2 was a 1-ice cream day (observation #0)
pr_obs_day_3 = b[:,2]   # day 3 was a 3-ice cream day (observation #2)

init_probabilities = np.array([0.8,0.2])

weather_likelihood_day_1   = init_probabilities
ice_cream_likelihood_day_1 = np.multiply( weather_likelihood_day_1 , pr_obs_day_1 )

weather_likelihood_day_2   = A @ ice_cream_likelihood_day_1
ice_cream_likelihood_day_2 = np.multiply( weather_likelihood_day_2 , pr_obs_day_2 )

weather_likelihood_day_3   = A @ ice_cream_likelihood_day_2
ice_cream_likelihood_day_3 = np.multiply( weather_likelihood_day_3 , pr_obs_day_3 )

# Each day includes "history", meaning summed probability of observations on all prior days

# The total probability of the observed sequence can be found by adding the probabilites at the end of day 3
print(
    "Likelihood of observing 3, 1, and 3 ice creams is: " + 
    f"{ice_cream_likelihood_day_3.sum():0.5f}")

Likelihood of observing 3, 1, and 3 ice creams is: 0.02856
